In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("colorblind")

%matplotlib inline

import datagenerators as dg

# Causal inference
# Part 1 : using the potential outcomes framework

We will give an overview of how we can use the [potential outcomes](https://en.wikipedia.org/wiki/Rubin_causal_model) framework to try and make causal inferences about situations where we only have observational data.

We will use generated data. These data are independent and identically distributed random variables from some distributions.

## Introduction: wearing cool hats

A manager notices that some membres of their team wear cool hats, and that these members tend to be less productive.

- hat: $X=1$ for a cool hat; $X=0$ for no cool hat
- productivity: $Y=1$ for productive; $Y=0$ for unproductive

After a week, they end up with the following dataset:

In [7]:
observed_data_0 = dg.generate_dataset_0()
print(observed_data_0.shape)
observed_data_0.head()

(500, 2)


,x,y
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0


## Are people wearing cool hats more likely to be productive that those who don't?
This is the first question the team lead asks.
This means estimating the quantity: 

$$P(Y=1|X=1) - (Y=1|X=0)$$

In [8]:
def estimate_uplift(ds):
    """
    Estimate the difference in means between two groups.
    
    Parameters
    ----------
    ds: pandas.DataFrame
        a dataframe of samples.
        
    Returns
    -------
    estimated_uplift: dict[Str: float] containing two items:
        "estimated_effect" - the difference in mean values of $y$ for treated and untreated samples.
        "standard_error" - 90% confidence intervals arround "estimated_effect"   
    """
    base = ds[ds.x == 0]
    variant = ds[ds.x == 1]

    delta = variant.y.mean() - base.y.mean()
    delta_err = 1.96 * np.sqrt(variant.y.var() / variant.shape[0] +
                               base.y.var() / base.shape[0])

    return {"estimated_effect": delta, "standard_error": delta_err}


estimate_uplift(observed_data_0)

{'estimated_effect': -0.16374269005847958,
 'standard_error': 0.08661600610290666}

It looks like people with cool hats are less productive.

To be sure, we can even run a statistical test:

In [12]:
from scipy.stats import chi2_contingency

contingency_table = observed_data_0.assign(placeholder=1).pivot_table(
    index="x", columns="y", values="placeholder", aggfunc="sum")

_, p, _, _ = chi2_contingency(contingency_table, lambda_="log-likelihood")
p

0.00034699294835294664

That's one small p-value. [Staticians would be proud](https://www.nature.com/articles/s41562-017-0189-z).

We can use this information to make statements about what we might think about someone's probability if we see them wearing a cool hat. As long as we believe that they are "drawn from the same distribution" as our previous observations, we expect the same correlations to exist.

The problem comes if we try to use this information as an argument about whether or not the team lead should **force** people to wear cool hats. If the team lead does this they fundamentally change the system we are sampling from, potentially altering or even reversing any correlations we observed before.

The cleanest way to actually measure the effect of some change in a system is by running a __randomized control trial__. 

Specifically, we want to randomize who gets cool hats and who doesn't, and look at the different values of $y$ we receive. This removes the effect of any [confounding variables](https://en.wikipedia.org/wiki/Confounding) which might be influencing the metric we care about.

Because we generated our dataset from a known process (in this case a function I wrote), we can intervene in it directly and measure the effect of an A/B test:

In [14]:
def run_ab_test(datagenerator, n_samples=10000, filter_=None):
    """
    Generates n_samples from datagenerator with the value of X randomized
    so that 50% of the samples receive treatment X=1 and 50% receive X=0,
    and feeds the results into `estimate_uplift` to get an unbiased 
    estimate of the average treatment effect.
    
    Returns
    -------
    effect: dict
    """
    n_samples_a = int(n_samples / 2)
    n_samples_b = n_samples - n_samples_a
    set_X = np.concatenate([np.ones(n_samples_a),
                            np.zeros(n_samples_b)]).astype(np.int64)
    ds = datagenerator(n_samples=n_samples, set_X=set_X)
    if filter_ != None:
        ds = ds[filter_(ds)].copy()
    return estimate_uplift(ds)

run_ab_test(dg.generate_dataset_0)

{'estimated_effect': 0.20140000000000002,
 'standard_error': 0.019200006338266608}

Wait. The direction of the effect has reversed?

## Definitions of causality

The previous example demonstrates the old statistics saying:

> Correlation does not imply causation.

In this notebook, causality mean "**What is the effect of $Y$ of changing $X$**".

To be precise:
- $X$ and $Y$ are random variables
- the _effect_ we want to know is __how the distribution of $Y$ will change when we take a certain value of $X$__?
- the act of forcing a variable to take a certain value is called an **intervention**

- when we make no intervention, we have an observational distribution of $Y$, conditioned on the fact we observe $X$:
$$P(Y|X)$$
- when we force people to wear cool hats, we are making an _intervention_. The distribution of $Y$ is given by the intervention distribution:
$$P(Y|\text{do}(X))$$

The question we will try to answer is how we can reason about the interventional distribution when we only have access to observational data.

This is a useful question because there are lots of situations where running an A/B test to directly measure the effects of an intervention is impractical, unfeasable or unethical.

In these situations we still want to be able to say something about what the effect of an intervention is - to do this we need to make some assumptions about the data generating process we are investigating.

## Potential outcomes

One way to approach this problem is to introduce two new random variables to our system: $Y_0$ and $Y_1$, known as the __potential outcomes__.
These variables exist but are never directly observed.

$Y$ is defined in terms of:

- $Y = Y_1$ when $X = 1$
- $Y = Y_0$ when $X = 0$

This shifts the problem from one about how distributions change under the intervention, to one about data drawn i.i.d. from some underlying distribution with [missing values](https://en.wikipedia.org/wiki/Missing_data).

## Goals

Often, we do not care about the full intervention distribution $P(Y|\text{do}(X))$, and it is enough to have an estimate of the difference in means.

This is a quantity known as the Average Treatment Effect (ATE):

$$\Delta = E[Y_{1} - Y_{0}]$$

When we run and A/B test and compare the means of each group, this is directly the quantity we are measuring.

If we just try and estimate this quantity from the observational distribution, we get:

$$
\begin{aligned}
\Delta_{bad} & = E[Y|X=1] - E[Y|X=0] \\
& = E[Y_{1}|X=1] - E[Y_{0}|X=0] \\
& \neq \Delta
\end{aligned}
$$

This is not generally equal to the true ATE because:

$E[Y_{i}|X=i] \neq E[Y_{i}]$

Two related quantities are 

 - $ATT = E[Y_{1} - Y_{0}|X=1]$, the "Average Treatment effect of the Treated": measure of the effect of treating only samples which would naturally be treated
 - $ATC = E[Y_{1} - Y_{0}|X=0]$, the "Average Treatment effect of the Control": measure of the effect of treating only samples which wouldn't naturally be treated

## Making assumptions